# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here
#pokemon.columns # 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 
#pokemon.dtypes

#is Total the sum of the other values?
pokemon[pokemon["Total"]==pokemon["HP"]+pokemon["Attack"]+pokemon["Defense"]+pokemon["Sp. Atk"]+pokemon["Sp. Def"]+pokemon["Speed"]]

#it seems so, there is an output. Is it true for all pokemon? Tested by checking if the sum is not equal to Total:
pokemon[pokemon["Total"]!=pokemon["HP"]+pokemon["Attack"]+pokemon["Defense"]+pokemon["Sp. Atk"]+pokemon["Sp. Def"]+pokemon["Speed"]]
#gives an empty result.
#Total is always the sum of the columns.

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [5]:
# your code here
#getting the new columns, but twice because of 2 types

pokemon_2typecols = pd.get_dummies(pokemon, columns = ["Type 1", "Type 2"], prefix='', prefix_sep='')
pokemon_2typecols

#merge columns with same names together
pokemon_typecols = pokemon_2typecols.groupby(level=0, axis=1).sum()
pokemon_typecols
##how? see here:
##https://stackoverflow.com/questions/70173154/group-and-sum-columns-that-have-the-same-name-in-different-cases-with-pandas

,#,Attack,Bug,Dark,Defense,Dragon,Electric,Fairy,Fighting,Fire,...,Normal,Poison,Psychic,Rock,Sp. Atk,Sp. Def,Speed,Steel,Total,Water
0,1,49,0,0,49,0,0,0,0,0,...,0,1,0,0,65,65,45,0,318,0
1,2,62,0,0,63,0,0,0,0,0,...,0,1,0,0,80,80,60,0,405,0
2,3,82,0,0,83,0,0,0,0,0,...,0,1,0,0,100,100,80,0,525,0
3,3,100,0,0,123,0,0,0,0,0,...,0,1,0,0,122,120,80,0,625,0
4,4,52,0,0,43,0,0,0,0,1,...,0,0,0,0,60,50,65,0,309,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,100,0,0,150,0,0,1,0,0,...,0,0,0,1,100,150,50,0,600,0
796,719,160,0,0,110,0,0,1,0,0,...,0,0,0,1,160,110,110,0,700,0
797,720,110,0,0,60,0,0,0,0,0,...,0,0,1,0,150,130,70,0,600,0
798,720,160,0,1,60,0,0,0,0,0,...,0,0,1,0,170,130,80,0,680,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [6]:
#preparing a list of all the types to get the right columns for .corr
pokemon_typecols.columns

#typelist
type1 = pokemon["Type 1"].unique()
type2 = pokemon["Type 2"].unique()
types = list(set(type1).union(set(type2)))
types = [item for item in types if not(pd.isnull(item)) == True]
types

['Ground',
 'Grass',
 'Electric',
 'Ice',
 'Dark',
 'Steel',
 'Fire',
 'Rock',
 'Ghost',
 'Poison',
 'Fighting',
 'Flying',
 'Psychic',
 'Dragon',
 'Bug',
 'Normal',
 'Water',
 'Fairy']

In [7]:
# filling a dict with all the Correlation values, turn it into a df

pokemon_typecols_dict = {t : pokemon_typecols['Total'].corr(pokemon_typecols[t]) for t in types}
pokemon_corr = pd.DataFrame(pokemon_typecols_dict, index=["corr"])   
pokemon_corr

,Ground,Grass,Electric,Ice,Dark,Steel,Fire,Rock,Ghost,Poison,Fighting,Flying,Psychic,Dragon,Bug,Normal,Water,Fairy
corr,0.01506,-0.052592,0.020971,0.060248,0.056154,0.109703,0.078726,0.032731,0.003641,-0.090441,0.077786,0.059383,0.124688,0.229705,-0.145781,-0.105331,-0.021665,-0.036698


In [83]:
#Rank the correlations and identify the #1 pokemon type that is most likely to have the highest Total
pokemon_corr.transpose().sort_values("corr", ascending=False)

#Solution:
#No 1 pokemon type is Dragon, showing the highest correlation with Total.

,corr
Dragon,0.229705
Psychic,0.124688
Steel,0.109703
Fire,0.078726
Fighting,0.077786
Ice,0.060248
Flying,0.059383
Dark,0.056154
Rock,0.032731
Electric,0.020971


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [9]:
def typecombo(row):
    if type(row["Type 2"]) == str:
        return row["Type 1"] +'-'+ row["Type 2"]
    else:
        return row["Type 1"]

pokemon["Combo Type"] = pokemon.apply(typecombo, axis=1)
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combo Type
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Grass-Poison
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Grass-Poison
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Grass-Poison
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,Grass-Poison
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,Fire
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,Fire
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,Fire-Flying
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,Fire-Dragon
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,Fire-Flying
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False,Water


In [16]:
pokemon_1typecol = pd.get_dummies(pokemon, columns = ["Combo Type"], prefix='type', prefix_sep='_')
#pokemon_1typecol.columns
filter_col = [col for col in pokemon_1typecol if col.startswith('type_')]
filter_col

['type_Bug',
 'type_Bug-Electric',
 'type_Bug-Fighting',
 'type_Bug-Fire',
 'type_Bug-Flying',
 'type_Bug-Ghost',
 'type_Bug-Grass',
 'type_Bug-Ground',
 'type_Bug-Poison',
 'type_Bug-Rock',
 'type_Bug-Steel',
 'type_Bug-Water',
 'type_Dark',
 'type_Dark-Dragon',
 'type_Dark-Fighting',
 'type_Dark-Fire',
 'type_Dark-Flying',
 'type_Dark-Ghost',
 'type_Dark-Ice',
 'type_Dark-Psychic',
 'type_Dark-Steel',
 'type_Dragon',
 'type_Dragon-Electric',
 'type_Dragon-Fairy',
 'type_Dragon-Fire',
 'type_Dragon-Flying',
 'type_Dragon-Ground',
 'type_Dragon-Ice',
 'type_Dragon-Psychic',
 'type_Electric',
 'type_Electric-Dragon',
 'type_Electric-Fairy',
 'type_Electric-Fire',
 'type_Electric-Flying',
 'type_Electric-Ghost',
 'type_Electric-Grass',
 'type_Electric-Ice',
 'type_Electric-Normal',
 'type_Electric-Steel',
 'type_Electric-Water',
 'type_Fairy',
 'type_Fairy-Flying',
 'type_Fighting',
 'type_Fighting-Dark',
 'type_Fighting-Flying',
 'type_Fighting-Psychic',
 'type_Fighting-Steel',
 'type_F

In [17]:
pokemon_1typecol.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,...,type_Water-Fighting,type_Water-Flying,type_Water-Ghost,type_Water-Grass,type_Water-Ground,type_Water-Ice,type_Water-Poison,type_Water-Psychic,type_Water-Rock,type_Water-Steel
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,0,0,0,0,0,0,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,0,0,0,0,0,0,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,0,0,0,0,0,0,0,0,0,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,...,0,0,0,0,0,0,0,0,0,0
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# filling a dict with all the Correlation values, turn it into a df

pokemon_typecol1_dict = {t : pokemon_1typecol['Total'].corr(pokemon_1typecol[t]) for t in filter_col}
pokemon_typecol1_dict

{'type_Bug': -0.17869896013466752,
 'type_Bug-Electric': -0.016537135952864844,
 'type_Bug-Fighting': 0.0479786775618784,
 'type_Bug-Fire': 0.008308759866728828,
 'type_Bug-Flying': -0.01736885054001315,
 'type_Bug-Ghost': -0.05875265075515327,
 'type_Bug-Grass': -0.0370537069849913,
 'type_Bug-Ground': -0.03762482904344436,
 'type_Bug-Poison': -0.08974237921758772,
 'type_Bug-Rock': -5.245410412691469e-05,
 'type_Bug-Steel': 0.05847146868724743,
 'type_Bug-Water': -0.04901476461650579,
 'type_Dark': 0.0010299475888800634,
 'type_Dark-Dragon': 0.0025062826825486633,
 'type_Dark-Fighting': -0.007141629130329428,
 'type_Dark-Fire': 0.02127035245150235,
 'type_Dark-Flying': 0.038960313305158645,
 'type_Dark-Ghost': -0.002130692158310529,
 'type_Dark-Ice': 0.014572431081752473,
 'type_Dark-Psychic': -0.020921705803381367,
 'type_Dark-Steel': -0.008394363373334145,
 'type_Dragon': -0.021410106187349197,
 'type_Dragon-Electric': 0.07226618092846718,
 'type_Dragon-Fairy': 0.04570830964124689,

In [19]:
pokemon_corr_combo = pd.DataFrame(pokemon_typecol1_dict, index=["corr"])   
pokemon_corr_combo

,type_Bug,type_Bug-Electric,type_Bug-Fighting,type_Bug-Fire,type_Bug-Flying,type_Bug-Ghost,type_Bug-Grass,type_Bug-Ground,type_Bug-Poison,type_Bug-Rock,...,type_Water-Fighting,type_Water-Flying,type_Water-Ghost,type_Water-Grass,type_Water-Ground,type_Water-Ice,type_Water-Poison,type_Water-Psychic,type_Water-Rock,type_Water-Steel
corr,-0.178699,-0.016537,0.047979,0.008309,-0.017369,-0.058753,-0.037054,-0.037625,-0.089742,-0.000052,...,0.06221,-0.024374,-0.011526,-0.045257,-0.001128,0.039182,-0.004317,0.030361,-0.003756,0.028003


In [20]:
#Rank the correlations and identify the #1 pokemon type that is most likely to have the highest Total
pokemon_corr_combo.transpose().sort_values("corr", ascending=False)

#Dragon+flying is the strongest combo

,corr
type_Dragon-Flying,0.149777
type_Dragon-Ice,0.128737
type_Dragon-Psychic,0.127066
type_Psychic-Fighting,0.104173
type_Ghost-Dragon,0.102264
...,...
type_Poison,-0.064380
type_Normal-Fairy,-0.069525
type_Bug-Poison,-0.089742
type_Normal-Flying,-0.092626
